## Laboration Statistiska metoder

### Introduktion

Jag kommer i denna labb göra en linjär regression på ett dataset samt tolka datan statistiskt.

### Metod

Jag har skapat en regressionsklass i Linear_regression.py, som hanterar alla beräkningar. I denna fil kommer jag kalla på klassens olika funktioner och visa resultaten. Datasetet som kommer utforskas är 'housing.csv', ett känt dataset som innehåller data om huspriser från Kalifornien 1997. Jag kommer göra två regressionsanalyser, en med endast numeriska kategorier och en där jag implementerar den kategoriska datan. Jag kommer även göra en 'one-hot encode' på den kategoriska datan, och lägga till denna i min regression för att sedan göra min analys.  
Innan detta görs gör jag en utforskning av datasetet, samt en enkel rensning. 

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
from pandas.api import types as ptypes
from Linear_regression import LinearRegression

df = pd.read_csv("../Data/housing.csv")
display(df.head())
is_num = df.dtypes.apply(ptypes.is_numeric_dtype)
numeric = is_num.sum()
categorical = len(is_num) - numeric

print(f'Antal kolumner: {df.columns.size} \nAntal numeriska kolumner: {numeric} \nAntal kategoriska kolumner: {categorical}')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Antal kolumner: 10 
Antal numeriska kolumner: 9 
Antal kategoriska kolumner: 1


In [2]:
display(df.isna().sum())
df_clean = df.dropna()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Datasetet består av 10 kolumner, varav 9 är numeriska och 1 är kategorisk, där den kategoriska är ocean_proximity. Det finns 207 NaN-värden i datasetet, alla befinner sig i kolumnen 'total_bedrooms'. Jag väljet att droppa dessa vilket ger mig 207 observationer mindre i mitt slutgiltiga dataset. 

In [3]:
df = df_clean.copy()
df['ocean_proximity'].value_counts()

ocean_proximity
<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: count, dtype: int64

Värt att notera här är att 'ISLAND' endast förekommer 5 gånger i datasetet, vilket framöver kommer göra den statistiskt opålitlig, och skapar även en obalans bland kategorierna. Jag väljer därför att slå ihop 'ISLAND' med 'NEAR OCEAN', då jag anser det vara ett geografiskt rimligt val. Jag kommer använda '<1H OCEAN' som min **baseline** då det är den mest förekommande kategorin.

In [4]:
df['ocean_proximity'] = df['ocean_proximity'].replace({'ISLAND': 'NEAR OCEAN'})
df['ocean_proximity'].value_counts()

ocean_proximity
<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2633
NEAR BAY      2270
Name: count, dtype: int64

## Första modellen

För att bygga min första regressionsmodell började jag med att analysera korrelationerna mellan de numeriska variablerna i datasetet med hjälp av min pearsonmetod. Syftet var att identifiera multikollinearitet — alltså variabler som i praktiken mäter samma sak och därför riskerar att skapa instabilitet i modellen. Jag väljer 'median_house_value' som min målvariabel, och resterande numeriska kategorier i min X-matris, som jag sedan skapar en korrelationsmatris av.

In [5]:
X = df_clean.iloc[:, :9].drop(columns=['median_house_value'], errors='ignore')
y = df_clean['median_house_value']

model = LinearRegression()
model.fit(X,y)

r = model.pearson_corr()
feature_names = df_clean.columns[:8]

corr_df = pd.DataFrame(r, columns=feature_names, index=feature_names)
corr_df


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
longitude,1.000000,-0.924616,-0.109357,0.045480,0.069608,0.100270,0.056513,-0.015550
latitude,-0.924616,1.000000,0.011899,-0.036667,-0.066983,-0.108997,-0.071774,-0.079626
housing_median_age,-0.109357,0.011899,1.000000,-0.360628,-0.320451,-0.295787,-0.302768,-0.118278
total_rooms,0.045480,-0.036667,-0.360628,1.000000,0.930380,0.857281,0.918992,0.197882
total_bedrooms,0.069608,-0.066983,-0.320451,0.930380,1.000000,0.877747,0.979728,-0.007723
population,0.100270,-0.108997,-0.295787,0.857281,0.877747,1.000000,0.907186,0.005087
households,0.056513,-0.071774,-0.302768,0.918992,0.979728,0.907186,1.000000,0.013434
median_income,-0.015550,-0.079626,-0.118278,0.197882,-0.007723,0.005087,0.013434,1.000000


In [6]:
corr = df_clean.iloc[:, :8].corr()

high_corr = (corr.abs().where(np.triu(np.ones(corr.shape), k=1).astype(bool)).stack())

high_corr = high_corr[high_corr > 0.8]
high_corr

longitude       latitude          0.924616
total_rooms     total_bedrooms    0.930380
                population        0.857281
                households        0.918992
total_bedrooms  population        0.877747
                households        0.979728
population      households        0.907186
dtype: float64

Analysen visade att **total_rooms, total_bedrooms, population** och **households** var mycket starkt korrelerade med varandra, ofta med korrelationskoefficienter över 0.9. Så höga värden innebär att variablerna i praktiken fångar samma underliggande fenomen, vilket också är logiskt: antalet rum och antalet sovrum hänger naturligt samman, och både total_rooms och population tenderar att öka i områden där antalet hushåll är större. Denna typ av överlapp gör variablerna överflödiga och kan dessutom skapa multikollinearitet, vilket försämrar modellens stabilitet och tolkbarhet.  

För att undvika detta valde jag att endast behålla **households**, som på ett mer sammanfattande och stabilt sätt representerar samma struktur. De övriga tre variablerna togs därför bort ur modellen.

Variablerna longitude och latitude uppvisade också en stark inbördes korrelation, men eftersom de representerar två olika geografiska dimensioner och båda är relevanta för bostadspriser valde jag att behålla dem.
Resultatet blev en första modell som enbart använder numeriska variabler och samtidigt undviker multikollinearitet.


In [7]:
df1 = df.drop(columns=['total_rooms', 'total_bedrooms', 'population', 'ocean_proximity'])
df1.head()

,longitude,latitude,housing_median_age,households,median_income,median_house_value
0,-122.23,37.88,41.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,259.0,3.8462,342200.0


In [11]:
X = df1.iloc[:, :5]
y = df1['median_house_value']

model.fit(X, y)
n = model.n
d = model.d

basic_summary = model.basic_summary()
basic_summary

,n,d,R2,Variance,Sigma,RMSE,F-statistic
0,20433,5,0.6,5.329792e+09,73005.42,72994.7,6131.3


## One-hot encoding

*Manuellt skapande av one hot encoding för den kategoriska variabeln.*

Jag använder en baseline‑kategori i one‑hot‑encoding för att undvika perfekt multikollinearitet. Om alla dummy‑variabler inkluderas samtidigt blir en av dem en exakt linjär kombination av de andra, vilket gör att regressionsmodellen inte kan beräkna koefficienterna. Genom att ta bort en kategori (baseline) blir modellen identifierbar och koefficienterna kan tolkas som skillnader gentemot baseline.

In [9]:
unique_cats = df['ocean_proximity'].unique().tolist()
cats_to_encode = [cat for cat in unique_cats if cat != '<1H OCEAN']

for cat in cats_to_encode:
    df[f'ocean_{cat}'] = (df['ocean_proximity'] == cat).astype(int)

df = df.drop(columns='ocean_proximity')
df_ocean = df[['ocean_NEAR BAY', 'ocean_INLAND', 'ocean_NEAR OCEAN']]
df


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_NEAR BAY,ocean_INLAND,ocean_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,1,0,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,1,0,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,1,0,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,1,0,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,0,1,0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,0,1,0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,0,1,0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,0,1,0


In [10]:
F, p = model.significance()
print(f"F = {F:.4f}, p = {p:.4e}")

alpha = 0.05
if p < alpha:
    print("Modellen är signifikant: förkasta H0 (minst en koefficient ≠ 0).")
else:
    print("Ingen signifikans: kan inte förkasta H0 (inga bevis att någon av koefficienterna ≠ 0).")

F = 6131.3008, p = 0.0000e+00
Modellen är signifikant: förkasta H0 (minst en koefficient ≠ 0).
